## Q1: I love pip 

- Take a piece of python code you wrote (for instance pick one of the exercises you've done for this class). 
- Turn it into a module
- Install it locally
- Deploy on pypi (only using the test-pypi server!)

<div class="alert alert-block alert-warning">
<span class="fa fa-flash"></span> VERY IMPORTANT
    
Both the pypi and test-pypi server are public on the web! Don't compromise your research by putting up something your competitors can use and steal your idea before you've published a paper on it!!! 

</div>




https://test.pypi.org/project/pip-test-ids-bicocca2025/0.0.3/

In [1]:
from pip_test_ids_bicocca2025 import Ids

N_MC = int(1e7)
batch = 100_000
N_core = 4

Ids = Ids.run_MC_sim(N_MC, batch, N_core)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 103.64it/s]


## Q4: It's over Anakin, I have the high ground.

Pick a piece of your code that your really hate. It's sooooo slooooow. 
- Profile it.
- Find the culprit
- Rewrite that bottleneck making a better use of numpy arrays, or jitting it with Numba.
- Enjoy your faster code

**You underestimate my power**.

In [2]:
import random
import cProfile
import matplotlib.pyplot as plt

%load_ext line_profiler

def montecarlo_ids(num_iter=int(1e6), Vgs=1.2, Vth_mean=0.7, Vth_sigma=0.05, k_mean=0.002, k_sigma=0.0002):

    ids_values = []
    W = 1e-6
    L = 180e-9
    lam = 1/20
    Vds = 1.1
    
    for i in range(num_iter):
        Vth_i = random.gauss(Vth_mean, Vth_sigma)
        k_i = random.gauss(k_mean, k_sigma)

        Id_i = 0.5*k_i*(Vgs-Vth_i)**2 *W/L * (1 + lam*Vds)

        ids_values.append(Id_i)
    return ids_values

In [3]:
%lprun -f montecarlo_ids montecarlo_ids()

Timer unit: 1e-07 s

Total time: 14.2259 s
File: C:\Users\carmi\AppData\Local\Temp\ipykernel_10040\1439846292.py
Function: montecarlo_ids at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def montecarlo_ids(num_iter=int(1e6), Vgs=1.2, Vth_mean=0.7, Vth_sigma=0.05, k_mean=0.002, k_sigma=0.0002):
     8                                           
     9         1         21.0     21.0      0.0      ids_values = []
    10         1         16.0     16.0      0.0      W = 1e-6
    11         1         13.0     13.0      0.0      L = 180e-9
    12         1         12.0     12.0      0.0      lam = 1/20
    13         1         13.0     13.0      0.0      Vds = 1.1
    14                                               
    15   1000001    6798290.0      6.8      4.8      for i in range(num_iter):
    16   1000000   74843612.0     74.8     52.6          Vth_i = random.gauss(Vth_mean, Vth_sigma)
    17   1000000   44186395

In [4]:
import numpy as np

def montecarlo_ids_eff(num_iter=int(1e6), Vgs=1.2, Vth_mean=0.7, Vth_sigma=0.05, k_mean=0.002, k_sigma=0.0002):

    W = 1e-6
    L = 180e-9
    lam = 1/20
    Vds = 1.1
    
    Vth = np.random.normal(Vth_mean, Vth_sigma, num_iter) 
    k   = np.random.normal(k_mean, k_sigma, num_iter)
    
    Id = 0.5 * k * (Vgs - Vth)**2 * (W/L) * (1 + lam*Vds) #switching from list to np.array for speed

    return Id


In [5]:
%lprun -f montecarlo_ids_eff montecarlo_ids_eff()

Timer unit: 1e-07 s

Total time: 0.0654822 s
File: C:\Users\carmi\AppData\Local\Temp\ipykernel_10040\1698860838.py
Function: montecarlo_ids_eff at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def montecarlo_ids_eff(num_iter=int(1e6), Vgs=1.2, Vth_mean=0.7, Vth_sigma=0.05, k_mean=0.002, k_sigma=0.0002):
     4                                           
     5         1         15.0     15.0      0.0      W = 1e-6
     6         1         13.0     13.0      0.0      L = 180e-9
     7         1         14.0     14.0      0.0      lam = 1/20
     8         1         11.0     11.0      0.0      Vds = 1.1
     9                                               
    10         1     270218.0 270218.0     41.3      Vth = np.random.normal(Vth_mean, Vth_sigma, num_iter) 
    11         1     237610.0 237610.0     36.3      k   = np.random.normal(k_mean, k_sigma, num_iter)
    12                                               

In this way it goes from nearly 4s to nearly 0.14s